# Importação das bibliotecas:

In [1]:
import glob
import pandas

# Listagem dos arquivos que devem ser analisados:

In [2]:
listaCsv = glob.glob('matrizPredecessor*.csv')

# Listagem dos códigos dos movimentos presentes nos arquivos:

In [3]:
listaCodigos = []

for i in range(len(listaCsv)):
    
    matrizPredecessor = pandas.read_csv(listaCsv[i], low_memory = False)

    listaCodigosTemp = list(matrizPredecessor['insercao'])
    
    listaCodigos = (listaCodigos + listaCodigosTemp)

listaCodigos = sorted(set(listaCodigos))

# Geração da Matriz de Contagem dos movimentos predecessores em relação a cada movimento na lista:

In [4]:
matrizContagem = pandas.DataFrame(0, index = range(len(listaCodigos)), columns = (['insercao'] + listaCodigos))

matrizContagem.loc[:, 'insercao'] = listaCodigos

matrizContagem.head()

,insercao,0,1,3,7,11,14,15,16,18,...,70345,83001,83002,100000,104663,200000,300000,400000,603103,99999999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
for i in range(len(listaCsv)):
    
    matrizPredecessor = pandas.read_csv(listaCsv[i], low_memory = False)
    
    listaCodigosTemp = list(matrizPredecessor['insercao'])
    
    for j in range(len(listaCodigosTemp)):
        
        for k in range(len(listaCodigosTemp)):
        
            insercao = listaCodigosTemp[j]
            
            anterior = listaCodigosTemp[k]

            linha = list(matrizContagem['insercao']).index(insercao)

            coluna = list(matrizContagem.columns).index(anterior)

            valor = matrizPredecessor.iloc[j, (k + 1)]

            matrizContagem.iloc[linha, coluna] = (matrizContagem.iloc[linha, coluna] + valor)

matrizContagem.head()

,insercao,0,1,3,7,11,14,15,16,18,...,70345,83001,83002,100000,104663,200000,300000,400000,603103,99999999
0,0,2392,0,30,0,0,0,0,18,16,...,2,0,0,0,0,0,0,0,0,0
1,1,0,23978,14152,0,0,31301,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,11,1268,5028,0,0,3575,0,31,10,...,0,0,0,0,0,1,0,21,0,0
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Exportação da Matriz de Contagem em formato CSV:

In [6]:
matrizContagem.to_csv('matrizContagem.csv', index = False)

# Geração da Matriz de Padrões:

In [7]:
colunasPadrao = ['insercao', 'primeiro', 'segundo', 'terceiro', 'quarto', 'quinto', 'sexto']

matrizPadrao = pandas.DataFrame(
    0, 
    index = range(len(listaCodigos)), 
    columns = colunasPadrao
)

matrizPadrao.loc[:, 'insercao'] = listaCodigos

matrizPadrao.head()

,insercao,primeiro,segundo,terceiro,quarto,quinto,sexto
0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,3,0,0,0,0,0,0
3,7,0,0,0,0,0,0
4,11,0,0,0,0,0,0


In [8]:
for i in range(len(listaCodigos)):

    listaValores = sorted(list(matrizContagem.loc[i])[1:], reverse = True)

    listaEntradas = list(matrizContagem.loc[i][1:])
    listaIndices = []

    for j in range(6):

        listaTemp = [k for k in range(len(listaEntradas)) if listaValores[j] == listaEntradas[k]]

        listaIndices = listaIndices + listaTemp

    listaIndices = list(dict.fromkeys(listaIndices))

    for j in range(6):
        
        if (listaValores[j] == 0):
            
            matrizPadrao.iloc[i, (j + 1)] = -1
            
        else:

            matrizPadrao.iloc[i, (j + 1)] = matrizContagem.columns[listaIndices[j] + 1]

matrizPadrao.head()

,insercao,primeiro,segundo,terceiro,quarto,quinto,sexto
0,0,0,132,246,11383,581,11010
1,1,14,1,51,3,493,132
2,3,51,3,85,132,14,48
3,7,51,-1,-1,-1,-1,-1
4,11,51,60,132,11010,-1,-1


# Exportação da Matriz de Padrões em formato CSV:

In [9]:
matrizPadrao.to_csv('matrizPadrao.csv', index = False)

# Exportação da Matriz de Padrões em formato JSON:

In [10]:
matrizPadrao.to_json('matrizPadrao.json', orient = 'records')